# Piline proyecto nivel 0 -1 taller 1

## Descripcion del dataset

In [2]:
# cargue librerias a utlizar 
import os
import requests
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from tensorflow_metadata.proto.v0 import schema_pb2
from tfx.v1.components import ImportSchemaGen
from tfx.types import artifact_utils
from tfx.types.standard_artifacts import Schema
import tensorflow_transform as tft
from tfx.components import Transform
#verifcar versiones de tf 
print('TF version:', tf.__version__)
print('TFDV version:', tfdv.version.__version__)


2025-03-03 18:20:11.877001: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-03 18:20:13.512118: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-03 18:20:14.395330: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-03 18:20:15.662447: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-03 18:20:15.666385: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-03 18:20:17.536243: I tensorflow/core/platform/cpu_feature_guard.cc:

TF version: 2.16.2
TFDV version: 1.16.1


In [3]:
# Directorio de los archivos de datos en bruto
_data_root = './data/covertype'
# Ruta al archivo de datos de entrenamiento en bruto
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')
# Crear el directorio si no existe
os.makedirs(_data_root, exist_ok=True)
# Verificar si el archivo no existe para descargarlo
if not os.path.isfile(_data_filepath):
    # URL del conjunto de datos (Repositorio UCI Machine Learning)
    url = 'https://docs.google.com/uc?export=download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'    
    # Realizar la solicitud HTTP para descargar el archivo con la opción de transmisión
    r = requests.get(url, allow_redirects=True, stream=True)
    # Abrir el archivo en modo escritura binaria y guardar el contenido directamente
    open(_data_filepath, 'wb').write(r.content)

In [21]:
# Check some data
df = pd.read_csv(_data_filepath)
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,119,7,67,11,1015,233,234,133,1570,Commanche,C7202,1
1,2876,3,18,485,71,2495,192,202,144,1557,Commanche,C7757,1
2,3171,315,2,277,9,4374,213,237,162,1052,Rawah,C7745,0
3,3087,342,13,190,31,4774,193,221,166,752,Rawah,C7745,0
4,2835,158,10,212,41,3596,231,242,141,3280,Rawah,C4744,1


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116203 entries, 0 to 116202
Data columns (total 13 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Elevation                           116203 non-null  int64 
 1   Aspect                              116203 non-null  int64 
 2   Slope                               116203 non-null  int64 
 3   Horizontal_Distance_To_Hydrology    116203 non-null  int64 
 4   Vertical_Distance_To_Hydrology      116203 non-null  int64 
 5   Horizontal_Distance_To_Roadways     116203 non-null  int64 
 6   Hillshade_9am                       116203 non-null  int64 
 7   Hillshade_Noon                      116203 non-null  int64 
 8   Hillshade_3pm                       116203 non-null  int64 
 9   Horizontal_Distance_To_Fire_Points  116203 non-null  int64 
 10  Wilderness_Area                     116203 non-null  object
 11  Soil_Type                           116

## Seleccion de caracteristicas

In [23]:
# se elimina los feactures no numericos ,'Wilderness_Area', 'Soil_Type'
df_numerico  =df.drop(['Wilderness_Area', 'Soil_Type'], axis=1)
df_numerico.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116203 entries, 0 to 116202
Data columns (total 11 columns):
 #   Column                              Non-Null Count   Dtype
---  ------                              --------------   -----
 0   Elevation                           116203 non-null  int64
 1   Aspect                              116203 non-null  int64
 2   Slope                               116203 non-null  int64
 3   Horizontal_Distance_To_Hydrology    116203 non-null  int64
 4   Vertical_Distance_To_Hydrology      116203 non-null  int64
 5   Horizontal_Distance_To_Roadways     116203 non-null  int64
 6   Hillshade_9am                       116203 non-null  int64
 7   Hillshade_Noon                      116203 non-null  int64
 8   Hillshade_3pm                       116203 non-null  int64
 9   Horizontal_Distance_To_Fire_Points  116203 non-null  int64
 10  Cover_Type                          116203 non-null  int64
dtypes: int64(11)
memory usage: 9.8 MB


In [ ]:
# separar datos en features y traget 
X = df_numerico.drop('Cover_Type', axis=1)  # eliminasmo el targe
y = df['Cover_Type']


In [26]:
selector_feactures = SelectKBest(f_classif, k=8) # las 8 carateristicas mas importante 
X_new = selector_feactures.fit_transform(X, y)



In [27]:
selected_features_main = X.columns[selector_feactures.get_support()]
most_relevant_features = {feat: (feat in selected_features_main) for feat in X.columns}

In [28]:
most_relevant_features

{'Elevation': True,
 'Aspect': False,
 'Slope': True,
 'Horizontal_Distance_To_Hydrology': True,
 'Vertical_Distance_To_Hydrology': True,
 'Horizontal_Distance_To_Roadways': True,
 'Hillshade_9am': True,
 'Hillshade_Noon': True,
 'Hillshade_3pm': False,
 'Horizontal_Distance_To_Fire_Points': True}

In [31]:
# Filtrar las claves con valor True
selected_features = [feature for feature, retain in most_relevant_features.items() if retain]

# Imprimir la lista de características seleccionadas
selected_features

['Elevation', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Horizontal_Distance_To_Fire_Points']


In [32]:
df_selected_features =df[list(selected_features) + 
                              ['Wilderness_Area', 'Soil_Type', 'Cover_Type']]#creamos nuevo dataframe con las features mas relevantes para y

In [33]:
df_selected_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116203 entries, 0 to 116202
Data columns (total 11 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Elevation                           116203 non-null  int64 
 1   Slope                               116203 non-null  int64 
 2   Horizontal_Distance_To_Hydrology    116203 non-null  int64 
 3   Vertical_Distance_To_Hydrology      116203 non-null  int64 
 4   Horizontal_Distance_To_Roadways     116203 non-null  int64 
 5   Hillshade_9am                       116203 non-null  int64 
 6   Hillshade_Noon                      116203 non-null  int64 
 7   Horizontal_Distance_To_Fire_Points  116203 non-null  int64 
 8   Wilderness_Area                     116203 non-null  object
 9   Soil_Type                           116203 non-null  object
 10  Cover_Type                          116203 non-null  int64 
dtypes: int64(9), object(2)
memory usage: 9.

In [34]:
df_selected_features.to_csv(_data_filepath,index=False)

## Data Pipeline

In [61]:
import tensorflow as tf

# Importación de componentes de TensorFlow Extended (TFX) para la canalización de datos.
from tfx.components import CsvExampleGen  # Componente para generar ejemplos a partir de archivos CSV.
from tfx.components import ExampleValidator # Componente para validar ejemplos contra un esquema de datos.
from tfx.components import SchemaGen       # Componente para generar un esquema de datos a partir de ejemplos.
from tfx.components import StatisticsGen   # Componente para generar estadísticas de los datos.
from tfx.components import Transform  

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

import os
import pprint
pp = pprint.PrettyPrinter()

In [63]:
_pipeline_root = './tfx_pipeline'
context = InteractiveContext(pipeline_root=_pipeline_root)

### Generando Ejemplos Con CsvExampleGen

In [64]:
# Instanciar el componente ExampleGen con el conjunto de datos CSV de entrada
example_gen = CsvExampleGen(input_base=_data_root)

In [65]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 5
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### Estadísticas

In [66]:
# Crear una elemento  de StatisticsGen con ExampleGen
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
# Ejecutar el componente
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 6
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [41]:
# Mostrar las estadísticas de salida
context.show(statistics_gen.outputs['statistics'])

### Inferir el esquema 

In [69]:
# Instanciar SchemaGen con el conjunto de datos procesado por StatisticsGen
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )

# Ejecutar el componente
context.run(schema_gen)



ExecutionResult(
    component_id: SchemaGen
    execution_id: 7
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [70]:
# Visualizar el esquema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,-
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


### Curando el esquema

In [72]:
schema_path = schema_gen.outputs['schema']._artifacts[0].uri + '/schema.pbtxt'
schema = tfdv.load_schema_text(schema_path)
# Define los dominios (rangos) para las características numéricas 'Hillshade_9am', 'Hillshade_Noon' y 'Slope'.
tfdv.set_domain(schema, 'Hillshade_9am', schema_pb2.IntDomain(name='Hillshade_9am', min=0, max=255)) # Establece el rango de 0 a 255 para 'Hillshade_9am'.
tfdv.set_domain(schema, 'Hillshade_Noon', schema_pb2.IntDomain(name='Hillshade_Noon', min=0, max=255)) # Establece el rango de 0 a 255 para 'Hillshade_Noon'.
tfdv.set_domain(schema, 'Slope', schema_pb2.IntDomain(name='Slope', min=0, max=90)) # Establece el rango de 0 a 90 para 'Slope'.

# Marca la característica 'Cover_Type' como categórica y define su rango de valores permitidos.
cover_type_feature = tfdv.get_feature(schema, 'Cover_Type') # Obtiene el objeto de característica para 'Cover_Type'.
cover_type_feature.int_domain.is_categorical = True # Marca 'Cover_Type' como categórica.
cover_type_feature.int_domain.min = 0 # Establece el valor mínimo permitido para 'Cover_Type'.
cover_type_feature.int_domain.max = 6 # Establece el valor máximo permitido para 'Cover_Type'.

tfdv.display_schema(schema=schema) # Muestra el esquema modificado.


,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


In [73]:
cover_type_feature.int_domain.is_categorical # Verifica que 'Cover_Type' se ha marcado como categórica.

True

### Entornos de esquema

In [81]:
# Configuración y validación de entornos de datos utilizando TensorFlow Data Validation (TFDV)

# 1. Configuración de entornos predeterminados para todas las características:
schema.default_environment.append('TRAINING')  # Agrega 'TRAINING' como entorno predeterminado.
schema.default_environment.append('SERVING')   # Agrega 'SERVING' como entorno predeterminado.
# Esto asegura que todas las características estén disponibles tanto en el entrenamiento como en el servicio, a menos que se especifique lo contrario.



In [82]:

# 2. Exclusión de la característica 'Cover_Type' del entorno de servicio:
tfdv.get_feature(schema, 'Cover_Type').not_in_environment.append('SERVING')
# Esto indica que 'Cover_Type' no se espera que esté presente en los datos de servicio.
tfdv.display_schema(schema=schema) # Muestra el esquema modificado.


,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


In [84]:

# 3. Verificación de entornos definidos en el esquema:
print('Entornos definidos en el esquema:', schema.default_environment)
# Imprime los entornos predeterminados configurados.

cover_type_feature = tfdv.get_feature(schema, 'Cover_Type')
if cover_type_feature:
    print('Entornos excluidos  Cover_Type:', cover_type_feature.not_in_environment)
else:
    print('el features Cover_Type no se encuentra en el esquema.')
# Verifica y muestra los entornos en los que 'Cover_Type' está excluida.



Entornos definidos en el esquema: ['TRAINING', 'SERVING', 'TRAINING', 'SERVING']
Entornos excluidos  Cover_Type: ['SERVING', 'SERVING']


In [85]:
# 4. Guardado del esquema modificado:
schema_path = os.path.join(_pipeline_root, 'schema.pptxt')
tfdv.write_schema_text(schema, schema_path)
# Guarda el esquema modificado en un archivo para su uso posterior.



In [87]:
# 5. Validación de estadísticas en diferentes entornos:
df = pd.read_csv(_data_filepath)
# Carga los datos desde un archivo CSV.

SERVING_DATA = df.drop('Cover_Type', axis=1)
# Simula datos de servicio eliminando la columna 'Cover_Type'.

serving_statistics = tfdv.generate_statistics_from_dataframe(SERVING_DATA)
# Genera estadísticas de los datos de servicio simulados.

anomalias = tfdv.validate_statistics(statistics=serving_statistics, schema=schema, environment='SERVING')
tfdv.display_anomalies(anomalias)
# Valida las estadísticas en el entorno 'SERVING' y muestra las anomalías.
# No se esperan anomalías debido a que 'Cover_Type' está excluida de este entorno.


In [88]:

anomalias = tfdv.validate_statistics(statistics=serving_statistics, schema=schema, environment='TRAINING')
tfdv.display_anomalies(anomalias)
# Valida las estadísticas en el entorno 'TRAINING' y muestra las anomalías.
# Se espera una anomalía debido a que 'Cover_Type' es requerida en este entorno.

,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',Column dropped,Column is completely missing


### Genere nuevas estadısticas usando el esquema actualizado

In [89]:
# Creación y ejecución de un nuevo artefacto de esquema y generación de estadísticas con el esquema importado.

# 1. Crear un nuevo artefacto que apunte al esquema anterior guardado
schema_new = ImportSchemaGen(schema_file=schema_path)
context.run(schema_new)

ExecutionResult(
    component_id: ImportSchemaGen
    execution_id: 8
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=ImportSchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [91]:

# 2. Generación de estadísticas utilizando el esquema importado
statistics_new = StatisticsGen(
    examples = example_gen.outputs['examples'],
    schema = schema_new.outputs['schema']
)

context.run(statistics_new)



ExecutionResult(
    component_id: StatisticsGen
    execution_id: 9
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [92]:

# 3. Mostrar las estadísticas generadas
context.show(statistics_new.outputs['statistics'])



### Comprobar anomalı́as

In [96]:
# Validación de ejemplos contra un esquema utilizando TensorFlow Data Validation (TFDV).
example_validator = ExampleValidator(
    statistics = statistics_new.outputs['statistics'],
    schema = schema_new.outputs['schema']
)
context.run(example_validator)
context.show(example_validator.outputs['anomalies'])

### Ingenierı́a de caracterı́sticas

In [97]:
example_gen.outputs['examples']

OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)

### Funciones de procesamiento

In [99]:
df

,Elevation,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,7,67,11,1015,233,234,1570,Commanche,C7202,1
1,2876,18,485,71,2495,192,202,1557,Commanche,C7757,1
2,3171,2,277,9,4374,213,237,1052,Rawah,C7745,0
3,3087,13,190,31,4774,193,221,752,Rawah,C7745,0
4,2835,10,212,41,3596,231,242,3280,Rawah,C4744,1
...,...,...,...,...,...,...,...,...,...,...,...
116198,3150,16,285,47,2275,200,253,866,Commanche,C7756,1
116199,3125,13,234,2,2430,224,212,1426,Rawah,C7745,0
116200,3166,11,67,0,1275,234,240,2404,Rawah,C7202,0
116201,3154,14,738,46,6012,181,239,1320,Rawah,C7745,1


In [103]:
df['Soil_Type'].unique()

array(['C7202', 'C7757', 'C7745', 'C4744', 'C7201', 'C7756', 'C2717',
       'C2706', 'C7746', 'C8772', 'C8771', 'C7102', 'C7755', 'C2703',
       'C4704', 'C2705', 'C2702', 'C7700', 'C8776', 'C4758', 'C4703',
       'C2704', 'C6731', 'C4201', 'C7701', 'C7702', 'C7710', 'C7101',
       'C6102', 'C8703', 'C7709', 'C6101', 'C3501', 'C8708', 'C3502',
       'C7790', 'C7103', 'C5101', 'C8707', 'C5151'], dtype=object)

In [104]:
df['Wilderness_Area'].unique()

array(['Commanche', 'Rawah', 'Neota', 'Cache'], dtype=object)

### Transformar

In [139]:
import preprocessing
transform = Transform(
    examples = example_gen.outputs['examples'],
    schema = schema_new.outputs['schema'],
    preprocessing_fn =  'preprocessing.preprocessing_fn'
)

context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 38
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [140]:
context.show(transform.outputs['post_transform_stats'])

## Metadatos de aprendizaje automático

In [143]:
from ml_metadata import metadata_store
from ml_metadata.proto import metadata_store_pb2

# Configura la conexión al almacén de metadatos
connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.sqlite.filename_uri = "./tfx_pipeline/metadata.sqlite"
connection_config.sqlite.connection_mode = 3  # READWRITE_OPENCREATE

store = metadata_store.MetadataStore(connection_config)

### Acceso a artefactos almacenados

In [165]:
#permite ver propiedades de lso shemas segun tipo y catidad de shemas que se desean ver
def inf_schema(schema_type_name, limit=3):
    # Obtener los esquemas del tipo dado
    schemas = store.get_artifacts_by_type(schema_type_name)

    # Limitar la cantidad de esquemas si se proporciona un límite
    if limit:
        schemas = schemas[:limit]

    # Iterar sobre los esquemas y mostrar sus propiedades
    for schema in schemas:
        print(f"ID del Esquema: {schema.id}")
        print("Propiedades:")
        for key, value in schema.custom_properties.items():
            if value.HasField('string_value'):
                print(f"  {key}: {value.string_value}")
            elif value.HasField('int_value'):
                print(f"  {key}: {value.int_value}")
        print("-" * 40)  # Separador visual entre esquemas


In [166]:
for artifact_type in store.get_artifact_types():
    print("////"*15)
    print(artifact_type.name)
    print("*****"*15)
    inf_schema(artifact_type.name)

////////////////////////////////////////////////////////////
Examples
***************************************************************************
ID del Esquema: 1
Propiedades:
  tfx_version: 1.16.0
  payload_format: FORMAT_TF_EXAMPLE
  file_format: tfrecords_gzip
  span: 0
  input_fingerprint: split:single_split,num_files:1,total_bytes:5542846,xor_checksum:1741027652,sum_checksum:1741027652
----------------------------------------
ID del Esquema: 4
Propiedades:
  tfx_version: 1.16.0
  payload_format: FORMAT_TF_EXAMPLE
  input_fingerprint: split:single_split,num_files:1,total_bytes:5542846,xor_checksum:1741027652,sum_checksum:1741027652
  span: 0
  file_format: tfrecords_gzip
----------------------------------------
ID del Esquema: 5
Propiedades:
  payload_format: FORMAT_TF_EXAMPLE
  file_format: tfrecords_gzip
  input_fingerprint: split:single_split,num_files:1,total_bytes:5542846,xor_checksum:1741027652,sum_checksum:1741027652
  span: 0
  tfx_version: 1.16.0
-------------------------

### Seguimiento de artefactos
     

In [225]:
def get_parent_artifacts(store, artifact_id):
    """
    Obtiene los artefactos de entrada que fueron utilizados para generar un artefacto en particular.

    """
    parent_artifacts_info = {}

    # Obtener eventos asociados al artefacto dado
    events = store.get_events_by_artifact_ids([artifact_id])

    # Extraer los execution_ids relacionados con estos eventos
    execution_ids = {event.execution_id for event in events}

    # Obtener eventos asociados a las ejecuciones anteriores
    parent_events = store.get_events_by_execution_ids(list(execution_ids))

    # Filtrar los artefactos de entrada (INPUT) usados en esas ejecuciones
    input_artifact_ids = {event.artifact_id for event in parent_events if event.type == event.INPUT}

    # Obtener los detalles de los artefactos de entrada
    parent_artifacts = store.get_artifacts_by_id(list(input_artifact_ids))

    for artifact in parent_artifacts:
        try:
            # Obtener el tipo de artefacto usando su type_id
            artifact_type = store.get_artifact_types_by_id([artifact.type_id])[0].name
        except IndexError:
            # Manejo de error en caso de no encontrar el tipo del artefacto
            print(f"Error: No se encontró tipo para artifact_id {artifact.id} con type_id {artifact.type_id}")
            continue

        # Guardar la información del artefacto padre en el diccionario
        parent_artifacts_info[artifact.id] = {
            'type': artifact_type,
            'uri': artifact.uri
        }

    return parent_artifacts_info



In [226]:
parent_artifacts = get_input_artifacts(store, 6)  # Reemplaza con un ID real
for artifact_id, info in parent_artifacts.items():
    print(f"ID: {artifact_id}, Tipo: {info['type']}, URI: {info['uri']}")


ID: 5, Tipo: Examples, URI: ./tfx_pipeline/CsvExampleGen/examples/5
ID: 6, Tipo: ExampleStatistics, URI: ./tfx_pipeline/StatisticsGen/statistics/6
